In [1]:
import torch
import torch.nn as nn
import numpy as np

from torch.utils.data import DataLoader,TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [2]:
features,target = make_classification(n_classes=2,
                                      n_features=10,
                                      n_samples=1000)

features_train,features_test,target_train,target_test = train_test_split(features,target,test_size=0.1,random_state=1)

torch.manual_seed(0)
np.random.seed(0)

x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1,1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1,1)

In [3]:
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet,self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,1),
            torch.nn.Dropout(0.1),
            torch.nn.Sigmoid(),
        )

    def forward(self,x):
        x = self.sequential(x)
        return(x)

In [4]:
network = SimpleNeuralNet()

criterion = nn.BCELoss()
optimizer = RMSprop(network.parameters())

train_data = TensorDataset(x_train,y_train)
train_loader = DataLoader(train_data,batch_size=100,shuffle=True)

epochs = 5
for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()

        torch.save(
            {
                'epoch' : epoch ,
                'model_state_dict' : network.state_dict(),
                'optimizer_state_dict' : optimizer.state_dict(),
                'loss' : loss,
            },
            'model.pt'
        )
    print("Epoch:", epoch+1,"\tLoss:" , loss.item())

Epoch: 1 	Loss: 0.3125213384628296
Epoch: 2 	Loss: 0.20736753940582275
Epoch: 3 	Loss: 0.2208128124475479
Epoch: 4 	Loss: 0.1908082515001297
Epoch: 5 	Loss: 0.19492369890213013
